In [1]:
import json
import requests

In [2]:
client_id = "..."
client_secret = "..."

In [3]:
class StepicAPI(object):
    def __init__(self, client_id, client_secret):
        self.api_url = 'https://stepic.org/api/'
        
        self.client_id = client_id
        self.client_secret = client_secret
        
        try:
            auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
            resp = requests.post('https://stepic.org/oauth2/token/',
                         data={'grant_type': 'client_credentials'},
                         auth=auth
                         )
            self.token = json.loads(resp.text)['access_token']
        except:
            print("Error while obtaining token")
      
    def get(self, url):
        try:
            resp = json.loads(requests.get(url, headers={'Authorization': 'Bearer ' + self.token}).text)
        except:
            print("Error while getting data")
            resp = None
        return resp
     
    # announcements
    def announcements(self, page = 1):
        url = self.api_url + 'announcements?page=%d'%page 
        return self.get(url)
    def announcements_pk(self, pk):
        url = self.api_url + 'announcements/%d'%pk 
        return self.get(url)
    
    # 
    def course_subscriptions(self, page = 1):
        url = self.api_url + 'course-subscriptions?page=%d'%page
        return self.get(url)
    def course(self, course_id):
        url = self.api_url + 'courses/%d'%course_id
        return self.get(url)
    
        

In [4]:
sapi = StepicAPI(client_id, client_secret)

### Пример получения списка курсов

In [5]:
has_next = True
page = 0
course_ids = []
while has_next:
    page += 1
    courses = sapi.course_subscriptions(page)
    has_next = courses['meta']['has_next']
    for el in courses['course-subscriptions']:
        course_ids.append(el['course'])
print("pages: %d"%page)    
print("course ids: %s"%str(course_ids))

pages: 2
course ids: [7, 67, 73, 75, 76, 83, 91, 95, 103, 104, 106, 114, 129, 144, 146, 150, 156, 187, 218, 326, 439, 501]


Для каждого курса получим информацию о нем

In [6]:
data = []
for course_id in course_ids:
    course_data = sapi.course(course_id)
    summary = (course_data['courses'][0]['summary'])
    title = (course_data['courses'][0]['title'])
    data.append((course_id, title, summary))

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(data, columns=['id','title','summary'])
df

,id,title,summary
0,7,Программирование на языке C++,Данный базовый курс повествует об основных сре...
1,67,Программирование на Python,Курс посвящен базовым понятиям и элементам язы...
2,73,Введение в Linux,Курс знакомит слушателей с операционной систем...
3,75,Функциональное программирование на языке Haskell,<br>Курс знакомит слушателей с языком Haskell...
4,76,Основы статистики,Курс знакомит слушателей с основными понятиями...
5,83,Дискретные структуры,Это базовый курс дискретной математики и комби...
6,91,Ликбез по дискретной математике,Это обзорный курс по дискретной математике. Ст...
7,95,Введение в математический анализ,Курс знакомит слушателей с базовыми понятиями ...
8,103,ФП CSC 2016,
9,104,Введение в теоретическую информатику,"Теоретическая информатика — раздел математики,..."
